<a href="https://colab.research.google.com/github/powidla/HSE_tasks/blob/main/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Чтобы научить алгоритм преодолевать лабиринт с помощью машинного обучения, можно использовать технику обучения с подкреплением, например Q-обучение. Обучение с подкреплением хорошо подходит для обучения агентов навигации в среде и принятию решений на основе проб и ошибок. Вот пошаговое руководство по обучению алгоритма с помощью Q-обучения:

1. Определите проблему: Определите характеристики и правила вашего лабиринта. Укажите начальную точку, местоположение цели, стены и возможные действия, которые может предпринять агент (например, двигаться вверх, вниз, влево, вправо).

2. Настройте пространство состояний: Определите представление лабиринта в виде сетки, где каждая ячейка может быть либо пустой, либо заблокированной стеной. Присвойте уникальные идентификаторы каждой ячейке, чтобы создать пространство состояний.

3. Инициализируйте Q-таблицу: Создайте Q-таблицу для хранения значений действий для каждой пары "состояние-действие". Q-таблица инициализируется нулями или случайными значениями.

4. Определите гиперпараметры: Определите скорость обучения (α), коэффициент дисконтирования (γ) и скорость исследования (ε). Эти гиперпараметры контролируют компромисс между эксплуатацией (выбор действий на основе полученных знаний) и исследованием (опробование новых действий).

5. Цикл обучения: Итерация по эпизодам для обучения агента. Каждый эпизод состоит из следующих шагов:

   a. Сброс среды: Поместите агента в начальную точку лабиринта.
   
   b. Выбрать действие: Используйте ε-жадную политику для выбора действия. С вероятностью ε выберите случайное действие; в противном случае выберите действие с наибольшим значением Q для текущего состояния.
   
   c. Выполнить действие: Переместите агента в соответствии с выбранным действием и наблюдайте за следующим состоянием и вознаграждением.
   
   d. Обновить Q-таблицу: Используйте правило обновления Q-обучения для обновления Q-значения для текущей пары "состояние-действие" на основе наблюдаемой награды и максимального Q-значения следующего состояния.
   
   e. Повторяйте шаги b-d, пока агент не достигнет цели или не застрянет в цикле.
   
   f. Обновить окружение: Если агент достиг цели, отметьте эпизод как успешный. В противном случае отметьте его как неудачный.
   
   g. Снизить скорость разведки: Снижайте скорость исследования ε со временем, чтобы стимулировать эксплуатацию.
   
   h. Повторяйте шаги a-g, пока значения Q не сойдутся или не будет достигнуто максимальное количество эпизодов.

6. Тестирование: После обучения оцените производительность обученного агента, позволив ему пройти лабиринт, используя выученную политику. Используйте самые высокие Q-значения для определения действий, которые необходимо предпринять в каждом состоянии.

7. Тонкая настройка: Если работа агента неудовлетворительна, скорректируйте гиперпараметры или измените структуру вознаграждения, чтобы улучшить процесс обучения.

8. Повторяйте шаги 5-7 до тех пор, пока агент не будет последовательно эффективно решать задачу лабиринта.

# Без визуализации

In [1]:
import numpy as np

# Задаем лабиринт как матрицу
labyrinth = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
])

# Определим стартовое положение агента и конечное положение которое будет считаться его конечной целью
start_pos = (0, 0)
goal_pos = (4, 4)

# Определим возможные варианты действия агента
actions = ['up', 'down', 'left', 'right']

# Инициализируем Q-таблицу
q_table = np.zeros((labyrinth.shape[0], labyrinth.shape[1], len(actions)))

# Параметры обучения
learning_rate = 0.1
discount_factor = 0.9
exploration_rate = 0.8
max_episodes = 1000

# Цикл обучения агента
for episode in range(max_episodes):
    # Положение агента
    current_pos = start_pos
    done = False

    while not done:
        # Условие остановки цикла
        if np.random.uniform() < exploration_rate:
            action = np.random.choice(actions)
        else:
            action = actions[np.argmax(q_table[current_pos])]

        # Подвигаем агента
        if action == 'up':
            next_pos = (current_pos[0] - 1, current_pos[1])
        elif action == 'down':
            next_pos = (current_pos[0] + 1, current_pos[1])
        elif action == 'left':
            next_pos = (current_pos[0], current_pos[1] - 1)
        elif action == 'right':
            next_pos = (current_pos[0], current_pos[1] + 1)

        if next_pos == goal_pos:
            reward = 1  # Успех
            done = True
        elif next_pos[0] < 0 or next_pos[0] >= labyrinth.shape[0] or next_pos[1] < 0 or next_pos[1] >= labyrinth.shape[1] or labyrinth[next_pos] == 1:
            reward = -1  # Врезался в стену
            next_pos = current_pos  # Остался на метсе
        else:
            reward = 0  # Пустая клетка

        # Обновление таблицы
        q_table[current_pos + (actions.index(action),)] += learning_rate * \
            (reward + discount_factor * np.max(q_table[next_pos]) - q_table[current_pos + (actions.index(action),)])

        current_pos = next_pos


Agent reached the goal!


In [2]:
# ПРоверка работы
%%time
current_pos = start_pos
done = False

while not done:
    action = actions[np.argmax(q_table[current_pos])]
    if action == 'up':
        next_pos = (current_pos[0] - 1, current_pos[1])
    elif action == 'down':
        next_pos = (current_pos[0] + 1, current_pos[1])
    elif action == 'left':
        next_pos = (current_pos[0], current_pos[1] - 1)
    elif action == 'right':
        next_pos = (current_pos[0], current_pos[1] + 1)

    if next_pos == goal_pos:
        done = True

    current_pos = next_pos

print("Agent reached the goal!")

Agent reached the goal!
CPU times: user 133 µs, sys: 0 ns, total: 133 µs
Wall time: 137 µs
